# CP101: Tutorial 17 Q1


Two fuel gases are mixed together prior to being burned with air in a
furnace to produce a flue gas with the following composition: CO$_2$
7%, CO 1%, O$_2$ 7% & N$_2$ 85% (on a dry basis).

The analyses of the fuel gases are as follows:
- Fuel Gas A: CH 4 80%, N 2 20%
- Fuel Gas B: CH 4 60%, C 2 H 6 20%, N 2 20%
All compositions are expressed on a mole basis.

Calculate: 
  (a) The mole ratio of Gas A to Gas B in the fuel gas mixture.
  (b) The percentage excess air used in the furnace.


## Atom balances

To begin solving the problem, we draw a process flow diagram for the furnace, labeling each of the streams and giving their overall flow rates and compositions.

![control volume](./CP101T17Q1.png)

We take $A$ to be the molar flow rate of fuel gas $A$, $B$ to be the molar flow rate of fuel gas $B$, $C$ to be the flow rate of air supplied to the furnace, $D$ to be molar flow rate of the flue gas from the furnace, excluding water, and $F_{\rm H_2O}$ to be the molar flow rate of water vapor in the flue gas.  

In this notebook, we will take a more general approach to solving this problem, with the goal to create a program that we can reuse for any problem of this type, regardless of the molecules or amounts involved.

First, in our calculations, we will take the convention where a postive flowrate corresponds to material entering the system, while a negative flowrate corresponds to material exiting the system.  Consequently, in the calculations, we take a basis where $\dot{N}_D=-100\,{\rm mol\,h^{-1}}$, where now $\dot{N}_S$ is the molar flow rate of stream $S$.

A balance over the element $e$ can be written in the general form:
\begin{align*}
\sum_{S\in\{A, B, C, D\} }  \sum_k n_{e,k} x_{k,S} \dot{N}_S 
&= 0
\end{align*}
where $x_{k,S}$ is the mole fraction of compound $k$ in stream $S$, $n_{e,k}$ is the number of atoms of element $e$ in a molecule of type $k$, and the index $k$ runs over all compounds in the system


In [ ]:
data = {}
data['CH4'] = {'C':1, 'H':4}
data['N2'] = {'N':2}
data['C2H6'] = {'C':2, 'H':6}
data['CO2'] = {'C':1, 'O':2}
data['CO'] = {'C':1, 'O':1}
data['O2'] = {'O':2}
data['H2O'] = {'H':2, 'O':1}


streamA = {}
streamA['dotN'] = 0.0
streamA['species'] = {'CH4':0.80, 'N2':0.20}

streamB = {}
streamB['dotN'] = 0.0
streamB['species'] = {'C2H6':0.20, 'CH4':0.6, 'N2':0.20}

streamC = {}
streamC['dotN'] = 0.0
streamC['species'] = {'N2':0.79, 'O2':0.21}


streamD = {}
streamD['dotN'] = -100.0
streamD['species'] = {'CO2':0.07, 'CO':0.01, 'O2':0.07,'N2':0.85,'H2O':0.0}


D = 100.0
A = 0.0
B = 0.0
C = 0.0
F = 0.0

streamA['dotN'] = A
streamB['dotN'] = B
streamC['dotN'] = C
streamD['dotN'] = D + F

streamD['species'] = {'CO2':0.07*D/(D+F), 'CO':0.01*D/(D+F), 'O2':0.07*D/(D+F),'N2':0.85*D/(D+F),'H2O':F/(D+F)}


res = []
for atom in ['N', 'C', 'O', 'H']:
    accumulation = 0.0
    print(atom + ' balance:')
    for stream in [streamA, streamB, streamC, streamD]:
        flowrate = stream['dotN']
        for molecule, x in stream['species'].items():
            if (atom in data[molecule]):
                natoms = data[molecule][atom]
                print(molecule, atom, natoms)
                accumulation += natoms*x*flowrate
        print(accumulation)
        res.append(accumulation)



## Python code

The element balances given above provide four equations which can be solved to determine the four unknowns: $A$, $B$, $C$, and $F_{\rm H_2O}$.  However, performing the algebra by hand is rather tedious.  Instead, we can use Python to solve these equations.


In [ ]:
from scipy.optimize import fsolve


def acc(x):
    
    D = 100.0
    A, B, C, F = x
    
    streamA['dotN'] = A
    streamB['dotN'] = B
    streamC['dotN'] = C
    streamD['dotN'] = -(D + F)

    streamD['species'] = {'CO2':0.07*D/(D+F), 'CO':0.01*D/(D+F), 'O2':0.07*D/(D+F),'N2':0.85*D/(D+F),'H2O':F/(D+F)}


    res = []
    for atom in ['N', 'C', 'O', 'H']:
        accumulation = 0.0
        for stream in [streamA, streamB, streamC, streamD]:
            flowrate = stream['dotN']
            for molecule, x in stream['species'].items():
                if (atom in data[molecule]):
                    natoms = data[molecule][atom]
                    accumulation += natoms*x*flowrate
        res.append(accumulation)
    
    return res



# make initial guess
A = 100.0
B = 0.0
C = 0.0
H2O_out = 0.0
x0 = [A, B, C, H2O_out]

# solve the equations
solution = fsolve(acc, x0)

print(solution)
